In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as bs
import pandas as pd
import os
import time

In [2]:
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)
pd.options.display.float_format = '{:.2f}'.format

In [3]:
service = Service(executable_path='c:\Temp\chromedriver.exe')

In [4]:
def get_comments(service, df, title_no, episode_no):
    TIMEOUT = 1   
    driver = webdriver.Chrome(service=service)
    
    # https://www.webtoons.com/en/action/omniscient-reader/episode-109/viewer?title_no=2154&episode_no=110
    url = 'https://www.webtoons.com/en/action/omniscient-reader/episode-109/viewer?'
    episode_url = url + 'title_no=' + str(title_no) + '&episode_no='+str(episode_no)
    print(episode_url)
    driver.get(episode_url)
    driver.implicitly_wait(TIMEOUT)
    
    # No iframe
   
    # No Cleanbot filtering setting

    # No need to click "전체 댓글"
    
    # Move to the last page
    driver.find_element(By.CSS_SELECTOR,'#cbox_module > div > div.u_cbox_paginate > div > a.u_cbox_next.u_cbox_next_end').click()
    time.sleep(1)
    
    # Get the total number of pages
    u_cbox_num_page = driver.find_element(By.CSS_SELECTOR,'#cbox_module > div > div.u_cbox_paginate > div > strong > span.u_cbox_num_page')
    time.sleep(0.5)
    num_pages = u_cbox_num_page.text
    print("num_pages:",num_pages)

    # Move to the first page
    driver.find_element(By.CSS_SELECTOR,'#cbox_module > div > div.u_cbox_paginate > div > a.u_cbox_pre.u_cbox_pre_end').click()
    time.sleep(1)
        
    # 0 ~ 446 
    for page in range(int(num_pages)):

        soup = bs(driver.page_source,"html.parser")

        # Extract dates
        dates = soup.findAll("span", {"class":["u_cbox_date"]})
        dates = [date.text for date in dates]        

        # Extract reveiws
        reviews = soup.findAll("span", {"class":["u_cbox_contents"]})
        reviews = [review.get_text() for review in reviews]

        df_page = pd.DataFrame(data=[], columns=['Episode','Date','Review'])       
        df_page['Date'] = dates
        df_page['Review'] = reviews
        df_page['Episode'] = df_page['Episode'].fillna(int(episode_no))
        
        df = pd.concat([df,df_page])      
        print(page+1, " page Done.")        

        if (page+1) == int(num_pages):
            return df
        
        # Move to the next page
        driver.find_element(By.CSS_SELECTOR, "#cbox_module > div > div.u_cbox_paginate > div > strong + a").click()        
        time.sleep(0.4)
    driver.close()
    return df

df = pd.DataFrame(data=[], columns=['Episode','Date','Review'])

# omniscient-reader - Episode 1
title_no = 2154
episode_no = 1
df = get_comments(service, df, title_no, episode_no)

# Save dataframe
DATA_PATH = './data/'
if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)
file_path = './data/omniscient-reader_Episode_{episode_no}.csv'.format(episode_no=episode_no)
df.to_csv(file_path,index = False)
print("Save Done.")
df

https://www.webtoons.com/en/action/omniscient-reader/episode-109/viewer?title_no=2154&episode_no=1
num_pages: 1
1  page Done.
Save Done.


,Episode,Date,Review
0,1,"Aug 19, 2020",This art is HONESTLY SPECTACULAR. Can't WAIT t...
1,1,"Aug 19, 2020",IT'S HERE!!!
2,1,"Aug 19, 2020","If he read the story over ten years, I hope he..."
3,1,"Aug 19, 2020",Yo this better become the next big action fant...
4,1,"Aug 19, 2020","DONT SLEEP ON THIS, THIS IS A TOP TIER MANHWA"
5,1,"Aug 19, 2020","i would read a 3,000 episode webtoon....just m..."
6,1,"Aug 19, 2020",This is one of the most popular korean webtoon...
7,1,"Aug 19, 2020",I’m a simple woman. I saw a hot guy on the ad....
8,1,"Aug 19, 2020",Um if I finish this is it gonna happen in real...
9,1,"Aug 19, 2020",Reminds me of Solo-Leveling!!


In [44]:
df

,Episode,Date,Review
0,1,"Aug 19, 2020",This art is HONESTLY SPECTACULAR. Can't WAIT t...
1,1,"Aug 19, 2020",IT'S HERE!!!
2,1,"Aug 19, 2020","If he read the story over ten years, I hope he..."
3,1,"Aug 19, 2020",Yo this better become the next big action fant...
4,1,"Aug 19, 2020","DONT SLEEP ON THIS, THIS IS A TOP TIER MANHWA"
...,...,...,...
14,1,"Sep 1, 2021","wait, sung jin woo????"
0,1,"Sep 3, 2020",Off brand ass solo leveling
1,1,"Jul 12, 2021",is this solo leveling?
2,1,"Aug 27, 2020",just a warning: prepare yourself for blood spl...
